In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import statistics
import sklearn.metrics as metrics
import tensorflow as tf
from sklearn.metrics import f1_score

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import RidgeCV, LassoCV, Ridge, Lasso
from sklearn.model_selection import RepeatedKFold
from sklearn.preprocessing import OneHotEncoder
from imblearn.over_sampling import SMOTE
from sklearn.feature_selection import RFE
from xgboost import XGBClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import SelectKBest, mutual_info_regression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import fbeta_score
from sklearn.feature_selection import RFECV
from imblearn.pipeline import Pipeline
from imblearn.under_sampling import RandomUnderSampler

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

from sklearn.ensemble import RandomForestClassifier
# from catboost import CatBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, roc_curve, auc, roc_auc_score
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

In [2]:
train = pd.read_csv("new_train.csv")
test = pd.read_csv("new_test.csv")

In [3]:
train

,Unnamed: 0,bidder_id,payment_account,address,outcome,auction,merchandise,device,time,country,...,max_url_per_auction,min_url_per_auction,std_url_per_auction,total_no_of_participated_auctions,no_of_auction_exceeds_threshold,percentage_of_auctions_above_threshold,total_no_of_bidded_category,no_of_merchandise_exceeds_threshold,percentage_of_merchandise_above_threshold,on_url_that_has_a_bot_mean
0,0,91a3c57b13234af24875c56fb7e2b2f4rb56a,a3d2de7675556553a5f08e4c88d2c228754av,a3d2de7675556553a5f08e4c88d2c228vt0u4,0.0,18.0,1.0,14.0,24.0,6.0,...,1.0,1.0,0.000000,18.0,0.0,0.000000,1.0,0.0,0.0,1.000000
1,1,624f258b49e77713fc34034560f93fb3hu3jo,a3d2de7675556553a5f08e4c88d2c228v1sga,ae87054e5a97a8f840a3991d12611fdcrfbq3,0.0,1.0,1.0,2.0,3.0,1.0,...,2.0,2.0,0.000000,1.0,0.0,0.000000,1.0,0.0,0.0,0.500000
2,2,1c5f4fc669099bfbfac515cd26997bd12ruaj,a3d2de7675556553a5f08e4c88d2c2280cybl,92520288b50f03907041887884ba49c0cl0pd,0.0,4.0,1.0,2.0,4.0,1.0,...,1.0,1.0,0.000000,4.0,0.0,0.000000,1.0,0.0,0.0,0.500000
3,3,4bee9aba2abda51bf43d639013d6efe12iycd,51d80e233f7b6a7dfdee484a3c120f3b2ita8,4cb9717c8ad7e88a9a284989dd79b98dbevyi,0.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,0.000000,1.0,0.0,0.000000,1.0,0.0,0.0,1.000000
4,4,4ab12bc61c82ddd9c2d65e60555808acqgos1,a3d2de7675556553a5f08e4c88d2c22857ddh,2a96c3ce94b3be921e0296097b88b56a7x1ji,0.0,23.0,1.0,53.0,155.0,2.0,...,21.0,1.0,5.644263,23.0,1.0,0.043478,1.0,0.0,0.0,0.010989
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2008,2008,369515b3af4f8ca582f90271d30b14b6r52aw,a1f85275793c4a782f0a668711f41b927ivc9,e6882cf204a9482edd042b6e31791dfctxzx8,0.0,25.0,1.0,4.0,33.0,4.0,...,1.0,1.0,0.000000,25.0,1.0,0.040000,1.0,0.0,0.0,0.500000
2009,2009,f939c17ffc7c39ac9b35b69e5e75179fv9pe2,a3d2de7675556553a5f08e4c88d2c2286s1m2,b9b03d5a127eb07aeb9163cdcf524e1344ac9,0.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,0.000000,1.0,0.0,0.000000,1.0,0.0,0.0,0.000000
2010,2010,c806dbb2decba0ed3c4ff5e2e60a74c2wjvbl,a3d2de7675556553a5f08e4c88d2c22856leq,d02c2b288b8aabd79ff47118aff41a2dqwzwc,0.0,1.0,1.0,2.0,2.0,1.0,...,1.0,1.0,0.000000,1.0,0.0,0.000000,1.0,0.0,0.0,0.000000
2011,2011,0381a69b7a061e9ace2798fd48f1f537mgq57,fd87037ce0304077079c749f420f0b4c54uo0,f030a221726fbcdfc4dc7dfd1b381a112hieq,0.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,0.000000,1.0,0.0,0.000000,1.0,0.0,0.0,0.000000


In [4]:
train.drop(train.filter(regex="Unname"),axis=1, inplace=True)
test.drop(test.filter(regex="Unname"),axis=1, inplace=True)

In [5]:
X = train.drop(columns=['bidder_id', 'payment_account', 'address', 'outcome','merchandise']) 
y = train['outcome']
X_test_original = test.drop(columns=['bidder_id', 'payment_account', 'address', 'merchandise'])

### Scaling the training and test dataset

In [6]:
X.shape

(2013, 52)

In [7]:
X.columns

Index(['auction', 'device', 'time', 'country', 'ip', 'url', 'num_bids',
       'num_first_bids', 'num_last_bids', 'time_to_bid', 'inst_resp',
       'perc_inst_resp', 'auto parts', 'books and music', 'clothing',
       'computers', 'furniture', 'home goods', 'jewelry', 'mobile',
       'office equipment', 'sporting goods', 'num_bids_per_auction',
       'num_bids_per_device', 'num_bids_per_country', 'num_bids_per_ip',
       'on_ip_that_has_a_bot_mean', 'ip_entropy', 'url_entropy',
       'mean_country_per_auction', 'max_country_per_auction',
       'min_country_per_auction', 'std_country_per_auction',
       'mean_devices_per_auction', 'max_devices_per_auction',
       'min_devices_per_auction', 'std_devices_per_auction',
       'mean_ip_per_auction', 'max_ip_per_auction', 'min_ip_per_auction',
       'std_ip_per_auction', 'mean_url_per_auction', 'max_url_per_auction',
       'min_url_per_auction', 'std_url_per_auction',
       'total_no_of_participated_auctions', 'no_of_auction_excee

In [8]:
scaler = StandardScaler()
# scaled_features = data.copy()
col_names = ['auction', 'device', 'time', 'country', 'ip', 'url', 'num_bids',
       'num_first_bids', 'num_last_bids', 'time_to_bid', 'inst_resp',
       'perc_inst_resp', 'num_bids_per_auction',
       'num_bids_per_device', 'num_bids_per_country', 'num_bids_per_ip',
       'on_ip_that_has_a_bot_mean', 'ip_entropy', 'url_entropy',
       'mean_country_per_auction', 'max_country_per_auction',
       'min_country_per_auction', 'std_country_per_auction',
       'mean_devices_per_auction', 'max_devices_per_auction',
       'min_devices_per_auction', 'std_devices_per_auction',
       'mean_ip_per_auction', 'max_ip_per_auction', 'min_ip_per_auction',
       'std_ip_per_auction', 'mean_url_per_auction', 'max_url_per_auction',
       'min_url_per_auction', 'std_url_per_auction',
       'total_no_of_participated_auctions', 'no_of_auction_exceeds_threshold',
       'percentage_of_auctions_above_threshold', 'total_no_of_bidded_category',
       'no_of_merchandise_exceeds_threshold',
       'percentage_of_merchandise_above_threshold',
       'on_url_that_has_a_bot_mean']

train_features = X[col_names]
scaler = StandardScaler().fit(train_features.values)
train_features = scaler.transform(train_features.values)
X[col_names] = train_features

test_features = X_test_original[col_names]
scaler_test = StandardScaler().fit(test_features.values)
test_features = scaler_test.transform(test_features.values)
X_test_original[col_names] = test_features

### Helper Functions

In [9]:
def evaluate(model, X_train, y_train, X_test, y_test):
    print("TRAIN")
    train_predictions = model.predict_proba(X_train)
    train_pred = model.predict(X_train)
        
    train_accuracy = accuracy_score(y_train, train_pred)
    train_auc_roc_score = roc_auc_score(y_train,train_predictions[:,1])
    train_fbeta = fbeta_score(y_train, train_pred, average='binary', beta=2.0)
    
    print("Classification report")
    print(classification_report(y_train, train_pred, digits = 4))
    
    print("FBeta Score")
    print(fbeta_score(y_train, train_pred, average='binary', beta=2.0))
    
    print('Model Performance')
    print('Accuracy = {:0.4f}%.'.format(train_accuracy))
    print('AUC ROC = {:0.4f}%.'.format(train_auc_roc_score))
    print("*" * 100)
    
    print("TEST")
    
    test_predictions = model.predict_proba(X_test)
    test_pred = model.predict(X_test)
        
    test_accuracy = accuracy_score(y_test, test_pred)
    test_auc_roc_score = roc_auc_score(y_test,test_predictions[:,1])
    test_fbeta = fbeta_score(y_test, test_pred, average='binary', beta=2.0)
    
    print("Classification report")
    print(classification_report(y_test, test_pred, digits = 4))
    
    print("FBeta Score")
    print(fbeta_score(y_test, test_pred, average='binary', beta=2.0))
    
    print('Model Performance')
    print('Accuracy = {:0.4f}%.'.format(test_accuracy))
    print('AUC ROC = {:0.4f}%.'.format(test_auc_roc_score))
    print("*" * 100)
    
    return [train_accuracy, train_auc_roc_score, train_fbeta, test_accuracy, test_auc_roc_score, test_fbeta]

In [10]:
def process_results(arr):
    train_accuracy = []
    train_auc_roc_score = [] 
    train_fbeta = []
    test_accuracy = []
    test_auc_roc_score = []
    test_fbeta = []
    
    for item in arr:
        train_accuracy.append(item[0])
        train_auc_roc_score.append(item[1])
        train_fbeta.append(item[2])
        test_accuracy.append(item[3])
        test_auc_roc_score.append(item[4])
        test_fbeta.append(item[5])
    
    mean_accuracy = np.array(train_accuracy).mean()
    mean_train_auc_roc_score = np.array(train_auc_roc_score).mean()
    mean_train_fbeta = np.array(train_fbeta).mean()
    mean_test_accuracy = np.array(test_accuracy).mean()
    mean_test_auc_roc_score = np.array(test_auc_roc_score).mean()
    mean_test_fbeta = np.array(test_fbeta).mean()
    
    print("final train accuracy: " + str(mean_accuracy))
    print("final train AUC: " + str(mean_train_auc_roc_score))
    print("final train fbeta: " + str(mean_train_fbeta))
    print("final test accuracy: " + str(mean_test_accuracy))
    print("final test AUC: " + str(mean_test_auc_roc_score))
    print("final test fbeta: " + str(mean_test_fbeta))

### SKFold + SMOTE (Base Model)

In [11]:
def create_model(learn_rate=0.01, momentum=0):
    # create model
    model = Sequential()
    model.add(Dense(64, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

skf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
counter = 1
dt_result = []
rf_result = []
xgb_result = []
ann_result = []

over = SMOTE(sampling_strategy=0.2, random_state = 42)
under = RandomUnderSampler(sampling_strategy=0.4, random_state = 42)
steps = [('o', over), ('u', under)]
pipeline = Pipeline(steps=steps)

for train_index, test_index in skf.split(X, y):
    print("Fold number " + str(counter))
    counter += 1
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    X_train, y_train = pipeline.fit_resample(X_train, y_train)
    
    # decision tree
    print("DECISION TREE")
    dt = DecisionTreeClassifier(random_state=42)
    dt.fit(X_train, y_train)
    dt_result.append(evaluate(dt, X_train, y_train, X_test, y_test))
    
    # random forest
    print("RANDOM FOREST")
    rf = RandomForestClassifier(random_state=42)
    rf.fit(X_train, y_train)
    rf_result.append(evaluate(rf,X_train,y_train,X_test,y_test))
    
    # xgboost
    print("XGBOOST")
    xgb = XGBClassifier(random_state=42)
    xgb.fit(X_train, y_train)
    xgb_result.append(evaluate(xgb, X_train, y_train, X_test, y_test))
    
    # ann
    print("ANN")
    ann = KerasClassifier(build_fn=create_model, verbose=0)
    ann.fit(X_train, y_train)
    ann_result.append(evaluate(ann, X_train, y_train, X_test, y_test))

Fold number 1
DECISION TREE
TRAIN
Classification report
              precision    recall  f1-score   support

         0.0     1.0000    1.0000    1.0000       762
         1.0     1.0000    1.0000    1.0000       305

    accuracy                         1.0000      1067
   macro avg     1.0000    1.0000    1.0000      1067
weighted avg     1.0000    1.0000    1.0000      1067

FBeta Score
1.0
Model Performance
Accuracy = 1.0000%.
AUC ROC = 1.0000%.
****************************************************************************************************
TEST
Classification report
              precision    recall  f1-score   support

         0.0     0.9694    0.9110    0.9393       382
         1.0     0.2273    0.4762    0.3077        21

    accuracy                         0.8883       403
   macro avg     0.5983    0.6936    0.6235       403
weighted avg     0.9307    0.8883    0.9064       403

FBeta Score
0.39062499999999994
Model Performance
Accuracy = 0.8883%.
AUC ROC = 0.6936%.


C:\Anaconda\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


TRAIN
Classification report
              precision    recall  f1-score   support

         0.0     1.0000    1.0000    1.0000       762
         1.0     1.0000    1.0000    1.0000       305

    accuracy                         1.0000      1067
   macro avg     1.0000    1.0000    1.0000      1067
weighted avg     1.0000    1.0000    1.0000      1067

FBeta Score
1.0
Model Performance
Accuracy = 1.0000%.
AUC ROC = 1.0000%.
****************************************************************************************************
TEST
Classification report
              precision    recall  f1-score   support

         0.0     0.9681    0.9529    0.9604       382
         1.0     0.3333    0.4286    0.3750        21

    accuracy                         0.9256       403
   macro avg     0.6507    0.6907    0.6677       403
weighted avg     0.9350    0.9256    0.9299       403

FBeta Score
0.4054054054054054
Model Performance
Accuracy = 0.9256%.
AUC ROC = 0.8761%.
*****************************

C:\Anaconda\lib\site-packages\tensorflow\python\keras\engine\sequential.py:425: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '
C:\Anaconda\lib\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '
C:\Anaconda\lib\site-packages\tensorflow\python\keras\engine\sequential.py:425: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.pred

Classification report
              precision    recall  f1-score   support

         0.0     0.9563    0.8586    0.9048       382
         1.0     0.1000    0.2857    0.1481        21

    accuracy                         0.8288       403
   macro avg     0.5281    0.5722    0.5265       403
weighted avg     0.9116    0.8288    0.8654       403

FBeta Score
0.20833333333333331
Model Performance
Accuracy = 0.8288%.
AUC ROC = 0.3611%.
****************************************************************************************************
Fold number 2
DECISION TREE
TRAIN
Classification report
              precision    recall  f1-score   support

         0.0     1.0000    1.0000    1.0000       762
         1.0     1.0000    1.0000    1.0000       305

    accuracy                         1.0000      1067
   macro avg     1.0000    1.0000    1.0000      1067
weighted avg     1.0000    1.0000    1.0000      1067

FBeta Score
1.0
Model Performance
Accuracy = 1.0000%.
AUC ROC = 1.0000%.
*****

...

Classification report
              precision    recall  f1-score   support

         0.0     0.9564    0.8613    0.9063       382
         1.0     0.0862    0.2500    0.1282        20

    accuracy                         0.8308       402
   macro avg     0.5213    0.5556    0.5173       402
weighted avg     0.9131    0.8308    0.8676       402

FBeta Score
0.18115942028985507
Model Performance
Accuracy = 0.8308%.
AUC ROC = 0.3267%.
****************************************************************************************************
Fold number 5
DECISION TREE
TRAIN
Classification report
              precision    recall  f1-score   support

         0.0     1.0000    1.0000    1.0000       762
         1.0     1.0000    1.0000    1.0000       305

    accuracy                         1.0000      1067
   macro avg     1.0000    1.0000    1.0000      1067
weighted avg     1.0000    1.0000    1.0000      1067

FBeta Score
1.0
Model Performance
Accuracy = 1.0000%.
AUC ROC = 1.0000%.
*****

C:\Anaconda\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


TRAIN
Classification report
              precision    recall  f1-score   support

         0.0     1.0000    1.0000    1.0000       762
         1.0     1.0000    1.0000    1.0000       305

    accuracy                         1.0000      1067
   macro avg     1.0000    1.0000    1.0000      1067
weighted avg     1.0000    1.0000    1.0000      1067

FBeta Score
1.0
Model Performance
Accuracy = 1.0000%.
AUC ROC = 1.0000%.
****************************************************************************************************
TEST
Classification report
              precision    recall  f1-score   support

         0.0     0.9785    0.9529    0.9655       382
         1.0     0.4000    0.6000    0.4800        20

    accuracy                         0.9353       402
   macro avg     0.6892    0.7764    0.7228       402
weighted avg     0.9497    0.9353    0.9414       402

FBeta Score
0.5454545454545454
Model Performance
Accuracy = 0.9353%.
AUC ROC = 0.8885%.
*****************************

C:\Anaconda\lib\site-packages\tensorflow\python\keras\engine\sequential.py:425: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '
C:\Anaconda\lib\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '
C:\Anaconda\lib\site-packages\tensorflow\python\keras\engine\sequential.py:425: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.pred

Classification report
              precision    recall  f1-score   support

         0.0     0.9759    0.7408    0.8423       382
         1.0     0.1161    0.6500    0.1970        20

    accuracy                         0.7363       402
   macro avg     0.5460    0.6954    0.5196       402
weighted avg     0.9331    0.7363    0.8102       402

FBeta Score
0.3385416666666667
Model Performance
Accuracy = 0.7363%.
AUC ROC = 0.6493%.
****************************************************************************************************


In [12]:
process_results(dt_result)

final train accuracy: 1.0
final train AUC: 1.0
final train fbeta: 1.0
final test accuracy: 0.8976679876053973
final test AUC: 0.7312341062079282
final test fbeta: 0.4435907933630869


In [13]:
process_results(rf_result)

final train accuracy: 1.0
final train AUC: 1.0
final train fbeta: 1.0
final test accuracy: 0.9319457304050467
final test AUC: 0.9000274245823985
final test fbeta: 0.5238738346001087


In [14]:
process_results(xgb_result)

final train accuracy: 0.9998125585754452
final train AUC: 0.9999995697259154
final train fbeta: 0.999475065616798
final test accuracy: 0.9359196572966433
final test AUC: 0.9051570680628271
final test fbeta: 0.5162981162981162


In [15]:
process_results(ann_result)

final train accuracy: 0.7057169634489222
final train AUC: 0.4865117679962136
final train fbeta: 0.4289991908914169
final test accuracy: 0.7819191881782157
final test AUC: 0.4752013213662429
final test fbeta: 0.2542838600029882


### SKFold + SMOTE (Random Search)

In [16]:
def random_search_dt(X, y):
    print("RANDOM SEARCH DECISION TREE EXPERIMENT")
    criterion = ['gini', 'entropy']
    max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
    max_depth.append(None)
    min_samples_split = [1,2,5,8]
    min_samples_leaf = [1,11,21,31]
    max_features = [5,10,15,25]
    min_impurity_decrease = [0.00005,0.0005,0.005,0.05]

    dt_random_grid = {'criterion': criterion,
                   'max_depth': max_depth,
                   'min_samples_split': min_samples_split,
                   'max_features': max_features,
                   'min_samples_leaf': min_samples_leaf, 
                   'min_impurity_decrease': min_impurity_decrease}
    
    
    skf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    over = SMOTE(sampling_strategy=0.2, random_state = 42)
    under = RandomUnderSampler(sampling_strategy=0.4, random_state = 42)
    steps = [('o', over), ('u', under)]
    pipeline = Pipeline(steps=steps)
    X_train, y_train = pipeline.fit_resample(X_train, y_train)
    
    
    dt = DecisionTreeClassifier(random_state=42)
    dt_random = RandomizedSearchCV(estimator = dt, param_distributions = dt_random_grid, n_iter = 100, cv = skf, 
                               verbose=2, random_state=42, n_jobs = -1, scoring='roc_auc')
    dt_random.fit(X_train, y_train)
    
    dt_best_random = dt_random.best_estimator_
    dt_random_accuracy = evaluate(dt_best_random, X_train, y_train, X_test, y_test)
    print("Best Random Search Param for DT")
    print(dt_random.best_params_)
    
def random_search_rf(X, y):
    print("RANDOM SEARCH RANDOM FOREST EXPERIMENT")
    # parameters for Random Forest
    n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
    max_features = ['auto', 'sqrt',5,10,25]
    max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
    max_depth.append(None)
    min_samples_split = [2, 5, 10]
    min_samples_leaf = [1, 2, 4,8,10]
    bootstrap = [True, False]

    # Create the random grid
    rf_random_grid = {'n_estimators': n_estimators,
                   'max_features': max_features,
                   'max_depth': max_depth,
                   'min_samples_split': min_samples_split,
                   'min_samples_leaf': min_samples_leaf,
                   'bootstrap': bootstrap}
    
    skf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    over = SMOTE(sampling_strategy=0.2, random_state = 42)
    under = RandomUnderSampler(sampling_strategy=0.4, random_state = 42)
    steps = [('o', over), ('u', under)]
    pipeline = Pipeline(steps=steps)
    X_train, y_train = pipeline.fit_resample(X_train, y_train)
    
    rf = RandomForestClassifier(random_state=42)
    rf_random = RandomizedSearchCV(estimator = rf, param_distributions = rf_random_grid, n_iter = 100, cv = skf, verbose=2, 
                               random_state=42, n_jobs = -1, scoring='roc_auc')
    rf_random.fit(X_train, y_train)
    
    rf_best_random = rf_random.best_estimator_
    rf_random_accuracy = evaluate(rf_best_random, X_train, y_train, X_test, y_test)
    print("Best Random Search Param for RF")
    print(rf_random.best_params_)
    
def random_search_xgb(X, y):
    print("RANDOM SEARCH XGB EXPERIMENT")
    xgb_random_grid = {'min_child_weight': range(1,10),
                    'gamma': [i/10.0 for i in range(0,10)],
                    'subsample': [i/10.0 for i in range(3,10)],
                    'colsample_bytree': [i/10.0 for i in range(6,10)],
                    'max_depth': [3, 4, 5]}
    
    skf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    over = SMOTE(sampling_strategy=0.2, random_state = 42)
    under = RandomUnderSampler(sampling_strategy=0.4, random_state = 42)
    steps = [('o', over), ('u', under)]
    pipeline = Pipeline(steps=steps)
    X_train, y_train = pipeline.fit_resample(X_train, y_train)

    
    xgb = XGBClassifier(random_state=42)
    xgb_random = RandomizedSearchCV(estimator = xgb, param_distributions = xgb_random_grid, n_iter=100, scoring='roc_auc', 
                                n_jobs=-1, cv=skf, random_state = 42)
    xgb_random.fit(X_train, y_train)
    xgb_best_random = xgb_random.best_estimator_
    xgb_random_accuracy = evaluate(xgb_best_random, X_train, y_train, X_test, y_test)
    print("Best Random Search Param for XGB")
    print(xgb_random.best_params_)
    
def random_search_ann(X, y):
    print("RANDOM SEARCH ANN EXPERIMENT")
    learn_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
    momentum = [0.0, 0.2, 0.4, 0.6, 0.8, 0.9]
    batch_size = [10, 20, 40, 60, 80, 100]
    epochs = [10, 50, 100]

    ann_random_grid = dict(learn_rate=learn_rate,momentum=momentum,batch_size=batch_size, epochs=epochs)
    
    skf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    over = SMOTE(sampling_strategy=0.2, random_state = 42)
    under = RandomUnderSampler(sampling_strategy=0.4, random_state = 42)
    steps = [('o', over), ('u', under)]
    pipeline = Pipeline(steps=steps)
    X_train, y_train = pipeline.fit_resample(X_train, y_train)

    ann = KerasClassifier(build_fn=create_model, verbose=0)
    ann_random = RandomizedSearchCV(estimator = ann, param_distributions = ann_random_grid, n_iter = 100, cv = skf, 
                                verbose=2, random_state=42, n_jobs = -1, scoring='roc_auc')
    ann_random.fit(X_train, y_train)
    ann_best_random = ann_random.best_estimator_
    ann_random_accuracy = evaluate(ann_best_random, X_train, y_train, X_test, y_test)
    print("Best Random Search Param for ANN")
    print(ann_random.best_params_)

In [17]:
random_search_dt(X, y)

RANDOM SEARCH DECISION TREE EXPERIMENT
Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    2.8s
[Parallel(n_jobs=-1)]: Done 346 tasks      | elapsed:    3.7s


TRAIN
Classification report
              precision    recall  f1-score   support

         0.0     0.9152    0.9414    0.9281       665
         1.0     0.8421    0.7820    0.8109       266

    accuracy                         0.8958       931
   macro avg     0.8787    0.8617    0.8695       931
weighted avg     0.8943    0.8958    0.8946       931

FBeta Score
0.7932875667429442
Model Performance
Accuracy = 0.8958%.
AUC ROC = 0.9650%.
****************************************************************************************************
TEST
Classification report
              precision    recall  f1-score   support

         0.0     0.9742    0.9151    0.9437       577
         1.0     0.2097    0.4815    0.2921        27

    accuracy                         0.8957       604
   macro avg     0.5919    0.6983    0.6179       604
weighted avg     0.9400    0.8957    0.9146       604

FBeta Score
0.38235294117647056
Model Performance
Accuracy = 0.8957%.
AUC ROC = 0.8106%.
*************

[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:    4.0s finished


In [18]:
random_search_rf(X, y)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


RANDOM SEARCH RANDOM FOREST EXPERIMENT
Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    5.7s
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed:   51.6s
[Parallel(n_jobs=-1)]: Done 341 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:  3.7min finished


TRAIN
Classification report
              precision    recall  f1-score   support

         0.0     0.9955    1.0000    0.9977       665
         1.0     1.0000    0.9887    0.9943       266

    accuracy                         0.9968       931
   macro avg     0.9978    0.9944    0.9960       931
weighted avg     0.9968    0.9968    0.9968       931

FBeta Score
0.9909570459683497
Model Performance
Accuracy = 0.9968%.
AUC ROC = 1.0000%.
****************************************************************************************************
TEST
Classification report
              precision    recall  f1-score   support

         0.0     0.9791    0.9723    0.9757       577
         1.0     0.4839    0.5556    0.5172        27

    accuracy                         0.9536       604
   macro avg     0.7315    0.7639    0.7464       604
weighted avg     0.9569    0.9536    0.9552       604

FBeta Score
0.5395683453237411
Model Performance
Accuracy = 0.9536%.
AUC ROC = 0.8943%.
**************

In [19]:
random_search_xgb(X, y)

RANDOM SEARCH XGB EXPERIMENT


C:\Anaconda\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[14:05:45] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
TRAIN
Classification report
              precision    recall  f1-score   support

         0.0     0.9970    1.0000    0.9985       665
         1.0     1.0000    0.9925    0.9962       266

    accuracy                         0.9979       931
   macro avg     0.9985    0.9962    0.9974       931
weighted avg     0.9979    0.9979    0.9978       931

FBeta Score
0.9939759036144578
Model Performance
Accuracy = 0.9979%.
AUC ROC = 1.0000%.
****************************************************************************************************
TEST
Classification report
              precision    recall  f1-score   support

         0.0     0.9807    0.9688    0.9747       577
         1

In [19]:
random_search_ann(X, y)

RANDOM SEARCH ANN EXPERIMENT
Fitting 5 folds for each of 100 candidates, totalling 500 fits


2021-11-07 17:04:10.548310: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-11-07 17:04:10.548310: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-11-07 17:04:10.548317: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the ap

TRAIN
Classification report
              precision    recall  f1-score   support

         0.0     0.8894    0.8827    0.8860       665
         1.0     0.7122    0.7256    0.7188       266

    accuracy                         0.8378       931
   macro avg     0.8008    0.8041    0.8024       931
weighted avg     0.8388    0.8378    0.8383       931

FBeta Score
0.7228464419475655
Model Performance
Accuracy = 0.8378%.
AUC ROC = 0.8592%.
****************************************************************************************************
TEST
Classification report
              precision    recall  f1-score   support

         0.0     0.9842    0.8614    0.9187       577
         1.0     0.1919    0.7037    0.3016        27

    accuracy                         0.8543       604
   macro avg     0.5880    0.7825    0.6101       604
weighted avg     0.9487    0.8543    0.8911       604

FBeta Score
0.45893719806763283
Model Performance
Accuracy = 0.8543%.
AUC ROC = 0.7871%.
*************


*****************************************************************************************
## Best Random Search Param for DT
{'min_samples_split': 2, 'min_samples_leaf': 11, 'min_impurity_decrease': 0.0005, 'max_features': 5, 'max_depth': 100, 'criterion': 'entropy'}

## Best Random Search Param for RF
{'n_estimators': 1000, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': 'auto', 'max_depth': 40, 'bootstrap': False}

## Best Random Search Param for XGB
{'subsample': 0.8, 'min_child_weight': 1, 'max_depth': 5, 'gamma': 0.1, 'colsample_bytree': 0.9}

## Best Random Search Param for ANN

### SKFold + SMOTE (Grid Search)

In [20]:
def grid_search_dt(X, y):
    print("GRID SEARCH DECISION TREE EXPERIMENT")
    dt_grid = {'criterion': ['entropy'],
        'max_depth': [90, 100, 110],
        'max_features': [5, 10, 15],
        'min_impurity_decrease': [0.0001, 0.0005, 0.001],
        'min_samples_leaf': [9,11,13],
        'min_samples_split': [1,2,3]
    }
    
    skf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    over = SMOTE(sampling_strategy=0.2, random_state = 42)
    under = RandomUnderSampler(sampling_strategy=0.4, random_state = 42)
    steps = [('o', over), ('u', under)]
    pipeline = Pipeline(steps=steps)
    X_train, y_train = pipeline.fit_resample(X_train, y_train)
    
    dt = DecisionTreeClassifier(random_state=42)
    dt_grid = GridSearchCV(estimator = dt, param_grid = dt_grid, cv = skf, n_jobs = -1, verbose = 2, scoring='roc_auc')
    dt_grid.fit(X_train, y_train)
    
    dt_best_grid = dt_grid.best_estimator_
    dt_grid_accuracy = evaluate(dt_best_grid, X_train, y_train, X_test, y_test)
    print("Best Params Grid Search for DT")
    print(dt_grid.best_params_)
    
    return dt_best_grid
    
def grid_search_rf(X, y):
    print("GRID SEARCH RANDOM FOREST EXPERIMENT")
    rf_grid = {
        'bootstrap': [False],
        'max_depth': [30,40,50],
        'max_features': ['auto'],
        'min_samples_leaf': [1,2,3],
        'min_samples_split': [1,2,3],
        'n_estimators': [900,1000,1100]
    }
    
    skf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    over = SMOTE(sampling_strategy=0.2, random_state = 42)
    under = RandomUnderSampler(sampling_strategy=0.4, random_state = 42)
    steps = [('o', over), ('u', under)]
    pipeline = Pipeline(steps=steps)
    X_train, y_train = pipeline.fit_resample(X_train, y_train)
    
    rf = RandomForestClassifier(random_state=42)
    rf_grid = GridSearchCV(estimator = rf, param_grid = rf_grid, cv = skf, n_jobs = -1, verbose = 2, scoring='roc_auc')
    rf_grid.fit(X_train, y_train)
    
    rf_best_grid = rf_grid.best_estimator_
    rf_grid_accuracy = evaluate(rf_best_grid, X_train, y_train, X_test, y_test)
    print("Best Params Grid Search for RF")
    print(rf_grid.best_params_)
    
    return rf_best_grid

def grid_search_xgb(X, y):
    print("GRID SEARCH XGB EXPERIMENT")
    xgb_grid = {
        'max_depth':[4,5,6],
        'min_child_weight':[1,2,3],
        'gamma':[0.1, 0.2, 0.3],
        'subsample':[0.7, 0.8, 0.9],
        'colsample_bytree':[0.8, 0.9, 1.0]
    }
    
    skf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    over = SMOTE(sampling_strategy=0.2, random_state = 42)
    under = RandomUnderSampler(sampling_strategy=0.4, random_state = 42)
    steps = [('o', over), ('u', under)]
    pipeline = Pipeline(steps=steps)
    X_train, y_train = pipeline.fit_resample(X_train, y_train)

    xgb = XGBClassifier(random_state=42, eval_metric="error")
    xgb_grid = GridSearchCV(estimator = xgb, param_grid = xgb_grid, scoring='roc_auc',n_jobs=-1, cv=skf)
    xgb_grid.fit(X_train, y_train)
    
    xgb_best_grid = xgb_grid.best_estimator_
    xgb_grid_accuracy = evaluate(xgb_best_grid, X_train, y_train, X_test, y_test)
    print("Best Params Grid Search for XGB")
    print(xgb_grid.best_params_)

    return xgb_best_grid

def grid_search_ann(X, y):
    learn_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
    momentum = [0.0, 0.2, 0.4, 0.6, 0.8, 0.9]
    batch_size = [10, 20, 40, 60, 80, 100]
    epochs = [10, 50, 100]
    
    ann_grid = dict(learn_rate=learn_rate,momentum=momentum,batch_size=batch_size, epochs=epochs)
    
    skf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    over = SMOTE(sampling_strategy=0.2, random_state = 42)
    under = RandomUnderSampler(sampling_strategy=0.4, random_state = 42)
    steps = [('o', over), ('u', under)]
    pipeline = Pipeline(steps=steps)
    X_train, y_train = pipeline.fit_resample(X_train, y_train)
    
    ann = KerasClassifier(build_fn=create_model, verbose=0)
    ann_grid = GridSearchCV(estimator = ann, param_grid = ann_grid, n_jobs=-1, cv=skf, scoring='roc_auc')
    ann_grid.fit(X_train, y_train)
    
    ann_best_grid = ann_grid.best_estimator_
    ann_grid_accuracy = evaluate(ann_best_grid, X_train, y_train, X_test, y_test)
    print("Best Params Grid Search for ANN")
    print(ann_grid.best_params_)
    return ann_best_grid

In [21]:
dt_best_grid = grid_search_dt(X, y)

GRID SEARCH DECISION TREE EXPERIMENT
Fitting 5 folds for each of 243 candidates, totalling 1215 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 987 tasks      | elapsed:    1.5s


TRAIN
Classification report
              precision    recall  f1-score   support

         0.0     0.9152    0.9414    0.9281       665
         1.0     0.8421    0.7820    0.8109       266

    accuracy                         0.8958       931
   macro avg     0.8787    0.8617    0.8695       931
weighted avg     0.8943    0.8958    0.8946       931

FBeta Score
0.7932875667429442
Model Performance
Accuracy = 0.8958%.
AUC ROC = 0.9650%.
****************************************************************************************************
TEST
Classification report
              precision    recall  f1-score   support

         0.0     0.9742    0.9151    0.9437       577
         1.0     0.2097    0.4815    0.2921        27

    accuracy                         0.8957       604
   macro avg     0.5919    0.6983    0.6179       604
weighted avg     0.9400    0.8957    0.9146       604

FBeta Score
0.38235294117647056
Model Performance
Accuracy = 0.8957%.
AUC ROC = 0.8106%.
*************

[Parallel(n_jobs=-1)]: Done 1215 out of 1215 | elapsed:    1.8s finished


In [22]:
rf_best_grid = grid_search_rf(X, y)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


GRID SEARCH RANDOM FOREST EXPERIMENT
Fitting 5 folds for each of 81 candidates, totalling 405 fits


[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    4.9s
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed:   38.2s
[Parallel(n_jobs=-1)]: Done 341 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 405 out of 405 | elapsed:  1.9min finished


TRAIN
Classification report
              precision    recall  f1-score   support

         0.0     0.9955    1.0000    0.9977       665
         1.0     1.0000    0.9887    0.9943       266

    accuracy                         0.9968       931
   macro avg     0.9978    0.9944    0.9960       931
weighted avg     0.9968    0.9968    0.9968       931

FBeta Score
0.9909570459683497
Model Performance
Accuracy = 0.9968%.
AUC ROC = 1.0000%.
****************************************************************************************************
TEST
Classification report
              precision    recall  f1-score   support

         0.0     0.9791    0.9723    0.9757       577
         1.0     0.4839    0.5556    0.5172        27

    accuracy                         0.9536       604
   macro avg     0.7315    0.7639    0.7464       604
weighted avg     0.9569    0.9536    0.9552       604

FBeta Score
0.5395683453237411
Model Performance
Accuracy = 0.9536%.
AUC ROC = 0.8934%.
**************

In [23]:
xgb_best_grid = grid_search_xgb(X, y)

GRID SEARCH XGB EXPERIMENT


C:\Anaconda\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


TRAIN
Classification report
              precision    recall  f1-score   support

         0.0     0.9970    1.0000    0.9985       665
         1.0     1.0000    0.9925    0.9962       266

    accuracy                         0.9979       931
   macro avg     0.9985    0.9962    0.9974       931
weighted avg     0.9979    0.9979    0.9978       931

FBeta Score
0.9939759036144578
Model Performance
Accuracy = 0.9979%.
AUC ROC = 1.0000%.
****************************************************************************************************
TEST
Classification report
              precision    recall  f1-score   support

         0.0     0.9773    0.9688    0.9730       577
         1.0     0.4375    0.5185    0.4746        27

    accuracy                         0.9487       604
   macro avg     0.7074    0.7437    0.7238       604
weighted avg     0.9531    0.9487    0.9507       604

FBeta Score
0.5
Model Performance
Accuracy = 0.9487%.
AUC ROC = 0.8895%.
*****************************

In [21]:
ann_best_grid = grid_search_ann(X, y)

TRAIN
Classification report
              precision    recall  f1-score   support

         0.0     0.9045    0.8827    0.8935       665
         1.0     0.7234    0.7669    0.7445       266

    accuracy                         0.8496       931
   macro avg     0.8139    0.8248    0.8190       931
weighted avg     0.8527    0.8496    0.8509       931

FBeta Score
0.7578008915304606
Model Performance
Accuracy = 0.8496%.
AUC ROC = 0.9052%.
****************************************************************************************************
TEST
Classification report
              precision    recall  f1-score   support

         0.0     0.9767    0.8700    0.9203       577
         1.0     0.1667    0.5556    0.2564        27

    accuracy                         0.8560       604
   macro avg     0.5717    0.7128    0.5883       604
weighted avg     0.9404    0.8560    0.8906       604

FBeta Score
0.3787878787878788
Model Performance
Accuracy = 0.8560%.
AUC ROC = 0.7055%.
**************

### Extract Feature Importance of each model

In [24]:
rf_feat_impt = rf_best_grid.feature_importances_
rf_features_rank = pd.DataFrame({"Features": X.columns, "rf_weights": rf_feat_impt})
rf_features_rank = rf_features_rank.sort_values(by=['rf_weights'], ascending=False)
rf_features_rank = rf_features_rank.reset_index().drop(columns=['index'])
rf_features_rank

,Features,rf_weights
0,num_bids_per_device,0.101344
1,num_bids_per_auction,0.093065
2,inst_resp,0.068570
3,perc_inst_resp,0.060903
4,num_bids,0.057389
5,num_bids_per_country,0.045099
6,time,0.041678
7,num_bids_per_ip,0.036461
8,no_of_auction_exceeds_threshold,0.036044
9,percentage_of_auctions_above_threshold,0.034575


In [25]:
dt_feat_impt = dt_best_grid.feature_importances_
dt_features_rank = pd.DataFrame({"Features": X.columns, "dt_weights": dt_feat_impt})
dt_features_rank = dt_features_rank.sort_values(by=['dt_weights'], ascending=False)
dt_features_rank = dt_features_rank.reset_index().drop(columns=['index'])
dt_features_rank

,Features,dt_weights
0,num_bids_per_country,0.555570
1,num_bids_per_ip,0.066281
2,time_to_bid,0.048772
3,time,0.039731
4,url_entropy,0.033945
5,mean_url_per_auction,0.030475
6,mean_devices_per_auction,0.028039
7,mean_ip_per_auction,0.026810
8,max_url_per_auction,0.022445
9,perc_inst_resp,0.022071


In [26]:
xgb_feat_impt = xgb_best_grid.feature_importances_
xgb_features_rank = pd.DataFrame({"Features": X.columns, "xgb_weights": xgb_feat_impt})
xgb_features_rank = xgb_features_rank.sort_values(by=['xgb_weights'], ascending=False)
xgb_features_rank = xgb_features_rank.reset_index().drop(columns=['index'])
xgb_features_rank

,Features,xgb_weights
0,num_bids_per_auction,0.213447
1,inst_resp,0.106258
2,num_bids_per_device,0.089635
3,num_bids,0.064537
4,computers,0.031721
5,mean_ip_per_auction,0.029687
6,percentage_of_auctions_above_threshold,0.025204
7,num_last_bids,0.024257
8,device,0.020187
9,num_bids_per_ip,0.020130


In [50]:
# rf_features_rank.to_csv('rf_features.csv', index=False)
# dt_features_rank.to_csv('dt_features.csv', index=False)
# xgb_features_rank.to_csv('xgb_features.csv', index=False)

In [36]:
skf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
over = SMOTE(sampling_strategy=0.2, random_state = 42)
under = RandomUnderSampler(sampling_strategy=0.4, random_state = 42)
steps = [('o', over), ('u', under)]
pipeline = Pipeline(steps=steps)
X_train, y_train = pipeline.fit_resample(X_train, y_train)

xgb_rfe = RFE(xgb_best_grid, n_features_to_select = 20)
xgb_rfe.fit(X_train, y_train)
xgb_features = xgb_rfe.support_
xgb_features_df = pd.DataFrame({"XGBoost features": X.columns, "selected":xgb_features})
# xgb_features_df
xgb_selected_features = xgb_features_df[xgb_features_df['selected'] == True]
xgb_selected_features

C:\Anaconda\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Anaconda\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Anaconda\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecat

C:\Anaconda\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Anaconda\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Anaconda\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecat

,XGBoost features,selected
2,time,True
3,country,True
4,ip,True
5,url,True
6,num_bids,True
7,num_first_bids,True
8,num_last_bids,True
9,time_to_bid,True
10,inst_resp,True
15,computers,True


In [35]:
rf_rfe = RFE(rf_best_grid, n_features_to_select = 20)
rf_rfe.fit(X_train, y_train)
rf_features = rf_rfe.support_
rf_features_df = pd.DataFrame({"Random Forest features": X.columns, "selected":rf_features})
# xgb_features_df
rf_selected_features = rf_features_df[rf_features_df['selected'] == True]
rf_selected_features

,Random Forest features,selected
2,time,True
4,ip,True
6,num_bids,True
9,time_to_bid,True
10,inst_resp,True
11,perc_inst_resp,True
22,num_bids_per_auction,True
23,num_bids_per_device,True
24,num_bids_per_country,True
25,num_bids_per_ip,True
